# Carregamento e preparação de *datasets*

O carregamento e preparação de *datasets* é um ótimo exercício para tomarmos conhecimento das ferramentas a serem utilizadas para o processamento de sinais em `python`, seja sinais biológicos quanto de outra natureza, como um som, corrente elétrica, etc.

Nesta `notebook` será apresentado o carregamento de um *dataset* público do *website* `UCI - Machine Learning Repository`. O *dataset* a ser utilizado é o `EEG Database Data Set` (https://archive.ics.uci.edu/ml/datasets/EEG+Database).


## Descrição do *dataset*:

A intenção deste *dataset* é examinar por meio de algoritmos de inteligência computacional a pré-disposição genética que um paciente possui ao alcoolismo.

Os principais dados analizados são do tipo *time-series*, em outras palavras, conjuntos de dados que representam um sinal mensurado no domínio do tempo. Os dados são completados com outros atributos como o nome do eletrodo, o número da amostra, etc. Outras informações relevantes do *dataset*:

- Quantidade de atributos: 4
- Número de instancias: 122
- Existem dados faltantes? Sim
- Tipos de dados encontrados: categórico, inteiro e real

Cada sessão (*trial*) é armazenada da seguinte forma:

```
# co2a0000364.rd 
# 120 trials, 64 chans, 416 samples 368 post_stim samples 
# 3.906000 msecs uV 
# S1 obj , trial 0 
# FP1 chan 0 
0 FP1 0 -8.921 
0 FP1 1 -8.433 
0 FP1 2 -2.574 
0 FP1 3 5.239 
0 FP1 4 11.587 
0 FP1 5 14.028
...
```

As primeiras 4 linhas são de cabeçalho:

**linha 1**: identificação do paciente e se ele indica ser um alcoólatra (a) ou controle (c) pela quarta letra (co2**a**0000364);

**linha 4**: determina se o paciente foi exposto a um único estímulo (`S1 obj`), a dois estímulos iguais (`S2 match`) ou a dois estímulos diferentes (`S2 no match`);

**linha 5**: identifica o início da coleta dos dados pelo eletrodo FP1. As 4 colunas são:

```
número_da_sessão identificação_do_eletrodo número_da_amostra valor_em_micro_volts
```


### Realizando o download 

Primeiro faremos um código para verificar se o *dataset* já foi baixado, caso contrário, executar o código de download:

In [3]:
from urllib.request import urlopen, urlretrieve
import os


urls = {
    'small': 'https://archive.ics.uci.edu/ml/machine-learning-databases/eeg-mld/smni_eeg_data.tar.gz',
    'large_train': 'https://archive.ics.uci.edu/ml/machine-learning-databases/eeg-mld/SMNI_CMI_TRAIN.tar.gz',
    'large_test': 'https://archive.ics.uci.edu/ml/machine-learning-databases/eeg-mld/SMNI_CMI_TEST.tar.gz',
    'full': 'https://archive.ics.uci.edu/ml/machine-learning-databases/eeg-mld/eeg_full.tar'
}

# verifica se o diretório dos datasets existe
if not os.path.exists('dataset/'):
    os.mkdir('dataset/')
    for k, v in urls.items():
        fn = v.split('/')[-1]
        print('Baixando:', fn, '...')
        urlretrieve(v, './dataset/{}'.format(fn))
    print('Downlod dos datasets concluído!')
else:
    print('Dataset já baixado!')

Dataset já baixado!


### Descompactando pastas e subpastas

Agora é necessário descompactar (recursivamente) diversas pastas e subpastas em arquivos GZip. Algumas pastas estão com o arquivo na extensão `.tar`, já outras, `.tar.gz`. Não obstante, algumas subpastas estão compactadas e outras não.

In [4]:
from subprocess import getoutput as gop
import glob

def descompacta():
    # único arquivo somente empacotado (tar)
    os.mkdir('dataset/eeg_full/')
    gop('tar -xvf dataset/eeg_full.tar -C dataset/eeg_full')
    os.remove('dataset/eeg_full.tar')

    while glob.glob('dataset/**/*.gz', recursive=True):
        # quando o arquivo está empacotado (tar) e compactado (gz)
        for f in glob.iglob('dataset/**/*.tar.gz', recursive=True):
            gop('tar -zxvf {} -C {}'.format(f, f[:f.rindex('/')]))
            os.remove(f)
        # quando o arquivo está somente compactado (gz)
        for f in glob.iglob('dataset/**/*.gz', recursive=True):
            gop('gzip -d {}'.format(f))
    print('Descompactações finalizadas!')

#descompata()

### Carregando parte do dataset

Vamos agora carregar o subconjunto "small" do *dataset* e fica como <font color='red'>**tarefa de casa**</font> carregar e preparar todos os outros subconjuntos...

In [7]:
def carrega():
    # organizando melhor as pastas
    os.rename('dataset/smni_eeg_data', 'dataset/small')
    os.rename('dataset/eeg_full', 'dataset/full')
    os.rename('dataset/SMNI_CMI_TRAIN/', 'dataset/large_train/')
    os.rename('dataset/SMNI_CMI_TEST/', 'dataset/large_test/')
    print(gop('ls -l dataset/'))

#carrega()

In [17]:
from re import search
import numpy as np

# identificando pastas
folders = {
    'small': 'dataset/small',
    'large_train': 'dataset/large_train',
    'large_test': 'dataset/large_test',
    'full': 'dataset/full',
}

def carrega():
    # carregando pasta "small"
    small_dir = gop('ls {}'.format(folders['small'])).split('\n')
    # 1ª dimensão dos dados contendo os sujeitos. Ex.: C_1, a_m, etc
    subjects = list()
    for types in small_dir:
        files = gop('ls {}/{}'.format(folders['small'], types)).split('\n')
        # 2ª dimensão dos dados contendo as sessões (trials)
        trials = list()
        for f in files:
            arquivo = open('{}/{}/{}'.format(folders['small'], types, f))
            text = arquivo.readlines()
            # 3ª dimensão dos dados contendo os canais (eletrodos)
            chs = list()
            # 4ª dimensão dos dados contendo os valores em milivolts
            values = list()
            for line in text:
                # ex: "# FP1 chan 0"
                t = search('\w{1,3} chan \d{1,2}', line)
                # ex: "0 FP1 0 -8.921"
                p = search('^\d{1,2}\ \w{1,3}\ \d{1,3}\ (?P<value>.+$)', line)
                if p:
                    values.append(float(p.group('value')))
                # mudou para outro eletrodo
                elif t and values:
                    chs.append(values)
                    values = list()
            chs.append(values)
            trials.append(chs)
            arquivo.close()
        subjects.append(trials)
    data = np.array(subjects)
    print(data.shape)

#carrega()

### Dados carregados...

Os dados "single" foram dividos da seguinte forma:
```
[experimentos, triagens, canais, amostras]
```
formando um `numpy.array` de quatro dimensões.

Em seguida, vamos plotar esses dados para "tentar" visualizar algum padrão.

In [12]:
%matplotlib inline

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

def carrega():
    d1 = list()
    d2 = list()

    for e in range(64):
        for i, t in enumerate(np.linspace(0, 1, 256)):
            d1.append([e, t, data[0][0][e][i]])
            d2.append([e, t, data[1][0][e][i]])
    d1 = np.array(d1)
    d2 = np.array(d2)
    x1, y1, z1 = d1[:,0], d1[:,1], d1[:,2]
    x2, y2, z2 = d2[:,0], d2[:,1], d2[:,2]

    fig = plt.figure()

    ax = fig.add_subplot(1, 2, 1, projection='3d')
    surf = ax.plot_trisurf(x1, y1, z1, cmap=cm.inferno, linewidth=1)
    ax.set_xlabel('Canais')
    ax.set_ylabel('Tempo (seg.)')
    ax.set_zlabel('Milivolts')

    ax = fig.add_subplot(1, 2, 2, projection='3d')
    surf = ax.plot_trisurf(x2, y2, z2, cmap=cm.inferno, linewidth=1)
    ax.set_xlabel('Canais')
    ax.set_ylabel('Tempo (seg.)')
    ax.set_zlabel('Milivolts')

    fig.colorbar(surf)
    fig.tight_layout()
    plt.show()

#carrega()

# Preparando ambiente

Para prosseguir, em cada sub-pasta do `large_test` e `large_train` será gerado três pastas para organizar os arquivos em suas respectivas categorias baseadas em tipo de teste, sendo elas: `s1`, `s2` e `s2no`.

In [14]:
import os
import shutil

def processaArquivo(nomepasta, file):
    for i in range(0, 3):
        file.readline()

    return file.readline().replace('#', '').split(',')[0].strip()

def copia(arquivos, nomepasta):
    for arquivo in arquivos:
        try:
            shutil.move(arquivo, nomepasta)
        except (FileExistsError, shutil.Error):
            return

def criaPastas(nomepasta, s1, s2, s2no):
    try:
        os.makedirs(nomepasta + 's1')
        os.makedirs(nomepasta + 's2')
        os.makedirs(nomepasta + 's2no')
    except FileExistsError:
        pass

    copia(s1, nomepasta + 's1/')
    copia(s2, nomepasta + 's2/')
    copia(s2no, nomepasta + 's2no/')

def organiza(nomepasta):
    s1 = []
    s2 = []
    s2no = []
    files = os.listdir(nomepasta)

    for fileName in files:
        if os.path.isdir(nomepasta + fileName):
            continue
        else:
            file = open(nomepasta + fileName)

        resp = processaArquivo(nomepasta, file)
        if 'S1' in resp:
            s1.append(nomepasta + fileName)
        else:
            if 'no' in resp:
                s2no.append(nomepasta + fileName)
            else:
                s2.append(nomepasta + fileName)

    return s1, s2, s2no

def listSubPaths(path):
    subPaths = []
    for subPath in os.listdir(path):
        if os.path.isdir(path + subPath):
            subPaths.append(path + subPath + '/')

    return subPaths

for path1 in ['large_test/', 'large_train/']:
    for path2 in listSubPaths('dataset/' + path1):
        s1, s2, s2no = organiza(path2)

        if s1 and s2 and s2no:
            criaPastas(path2, s1, s2, s2no)

As funções a seguir se referem as manipulações dos dados no domínio da frequência.

In [13]:
import os
import mne
import json
import random
import numpy as np
import matplotlib.pyplot as plt
from mne import set_eeg_reference as car

def toCSV(psds, filename, label, writeMode):
    file = open(filename, writeMode)
    for line in psds:
        line = list(line)
        for value in line:
            file.write(str(value) + ';')

    file.write('{};\n'.format(label))
    file.close()

def semMedia(pathName):
    files = os.listdir(pathName)
    resp =[]
    for i,file in enumerate(files):
        dic = {}
        processFile2(open(pathName + file), dic)
        resp.append(dic)

    return resp

def getSumElectodes(pathName):
    files = os.listdir(pathName)
    qtdElectrodes = len(files)
    resp = {}
    for file in files:
        processFile(open(pathName + file), resp, qtdElectrodes)

    return resp

def processFile2(file, resp):
    for line in file.readlines():
        if line.startswith('#'):
            continue
        # 41 FP2 168 -4.720
        line = line.split()
        # recupera os valores da linha
        electrode, pos, value = line[1:4]
        insert2(resp, electrode, int(pos), float(value))

def processFile(file, resp, qtdElectrodes):
    # para cada linha do arquivo
    for line in file.readlines():
        if line.startswith('#'):
            continue
        # 41 FP2 168 -4.720
        line = line.split()
        # recupera os valores da linha
        electrode, pos, value = line[1:4]
        insert(resp, electrode, int(pos), float(value), qtdElectrodes)

def insert2(resp, electrode, pos, value):
    try:
        resp[electrode][pos] = value
    except KeyError:
        resp[electrode] = [0]*256
        insert2(resp, electrode, pos, value)

def insert(resp, electrode, pos, value, qtdElectrodes):
    try:
        resp[electrode][pos] += (value*(1/qtdElectrodes))
    except KeyError:
        resp[electrode] = [0]*256

        insert(resp, electrode, pos, value, qtdElectrodes)

def viewTemporal(raw):
    # Neste primeiro gráfico mostramos o sinal de um eletrodo no domínio do tempo
    plt.plot(np.linspace(0, 1, 256), raw.get_data()[0])
    plt.xlabel('tempo (s)')
    plt.ylabel('Dados EEG (mV/cm²)')

def viewFrequencia(raw):
    raw.plot_psd()

def applyCar(raw):
    chs_P = ['CP5', 'C3', 'CP6', 'C4']
    inst, data = car(raw, ref_channels=chs_P)
    return inst

def plotaGrafico(electrodes):
    dataInput = []
    ch_types = ['eeg'] * 64
    ch_names = list(electrodes.keys())

    for electrode in ch_names:
        dataInput.append(electrodes[electrode])

    info = mne.create_info(ch_names=ch_names, sfreq=256, ch_types=ch_types)
    raw = mne.io.RawArray(dataInput, info)
    # São removidos aqui alguns canais que não parecem ser informações de eletrodos EEG
    raw.drop_channels(['X', 'nd', 'Y'])
    # Aplicamos a montagem do padrão 10-20 para todos os eletrodos
    montage = mne.channels.read_montage('standard_1020')
    raw.set_montage(montage)
    # Aqui mostramos todos os 61 eletrodos que representam dados EEG
    raw2 = applyCar(raw)
    return raw2

Os códigos a seguir realizam o pré-processamento dos dados, sendo este dividido em duas etapas:
    - utiliando a média de cada teste por pessoa (20 entradas); e
    - utilizando sem a média (200 entradas).

In [15]:
from mne.time_frequency import psd_welch as pw
import numpy

def person(pathName):
    if 'a' in pathName:
        return 'alcohol'
    else:
        return 'control'

categorys = ['train', 'test']
mainPath = 'dataset/'

pathsS = ['s1', 's2', 's2no']

for category in categorys:
    secondPath = 'large_{}/'.format(category)
    for s in pathsS:
        open(mainPath + category + '_' + s.replace('/', '') + '.csv', 'w').close()

    for path in os.listdir(mainPath + secondPath):
        print('.', end='', flush=True)
        if os.path.isfile(mainPath + secondPath + path):
            continue

        path += '/'

        for s in pathsS:
            s += '/'
            sMedia = semMedia(mainPath + secondPath + path + s)

            for sM in sMedia:
                psds, freqs = pw(plotaGrafico(sM))

                #toCSV(psds[int(len(psds)/2):], mainPath + category + '_' + s.replace('/', '') + '.csv', person(path), 'a')            
                toCSV(psds, mainPath + category + '_' + s.replace('/', '') + '.csv', person(path), 'a')            

            # media = getSumElectodes(mainPath + secondPath + path + s)
            # psds, freqs = pw(plotaGrafico(media))
            # toCSV(psds[int(len(psds)/2):], mainPath + category + '_' + s.replace('/', '') + '.csv', person(path), 'a')

..........................................

# Classificador

Para cada arquivo categoria de teste aplicada no usuário, é realizado a classificação por dois algoritmos supervisionado, sendo eles o `KNN` e o `SVM`.

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler, PowerTransformer

def extrai(fileName):
    x, y = [], []
    file = open(fileName)
    # pre processing POWER
    power = PowerTransformer()
    for line in file.readlines():
        line = line.replace(';\n', '').split(';')
        y.append(line.pop())
        x.append(line)

    return power.fit_transform(np.array(x)), np.array(y)

def printResult(train_values, test_values, acertos, k=False):
    if k:
        print('KNN:', k)
    else:
        print('SVM:')

    print('Total de treinamento: %d' % len(train_values))
    print('Total de testes: %d' % (len(test_values)))
    print('Total de acertos: %d' % acertos)
    print('Porcentagem de acertos: %.2f%%' % (100 * acertos / (len(test_values))))
    print()

def knn(k):
    global train_values
    global test_values

    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(train_values, train_label)
    rotulos_previstos = knn.predict(test_values)
    acertos, indice_rotulo = 0, 0
    for i in range(0, len(test_values)):
        if rotulos_previstos[indice_rotulo] == test_label[i]:
            acertos += 1
        indice_rotulo += 1

    printResult(train_values, test_values, acertos, k)

def svmPSB():
    global train_values
    global test_values

    xTrain = train_values
    xTest = test_values

    clf = svm.SVC(kernel='linear')
    clf.fit(xTrain, train_label)
    rotulos_previstos = clf.predict(test_values)
    # Contabilizar acertos para os dados de teste
    acertos, indice_rotulo = 0, 0
    for i in range(0, len(test_values)):
        if rotulos_previstos[indice_rotulo] == test_label[i]:
            acertos += 1
        indice_rotulo += 1

    printResult(train_values, test_values, acertos)

for s in ['s1', 's2', 's2no']:
    print("Category: {}".format(s.upper()))

    fileNameTest = './dataset/test_{}.csv'.format(s)
    fileNameTrain = './dataset/train_{}.csv'.format(s)

    test_values, test_label = extrai(fileNameTest)
    train_values, train_label = extrai(fileNameTrain)

    knn(9)
    knn(7)
    knn(5)

    svmPSB()

Category: S1
KNN: 9
Total de treinamento: 200
Total de testes: 200
Total de acertos: 196
Porcentagem de acertos: 98.00%

KNN: 7
Total de treinamento: 200
Total de testes: 200
Total de acertos: 195
Porcentagem de acertos: 97.50%

KNN: 5
Total de treinamento: 200
Total de testes: 200
Total de acertos: 194
Porcentagem de acertos: 97.00%

SVM:
Total de treinamento: 200
Total de testes: 200
Total de acertos: 193
Porcentagem de acertos: 96.50%

Category: S2
KNN: 9
Total de treinamento: 200
Total de testes: 200
Total de acertos: 194
Porcentagem de acertos: 97.00%

KNN: 7
Total de treinamento: 200
Total de testes: 200
Total de acertos: 196
Porcentagem de acertos: 98.00%

KNN: 5
Total de treinamento: 200
Total de testes: 200
Total de acertos: 196
Porcentagem de acertos: 98.00%

SVM:
Total de treinamento: 200
Total de testes: 200
Total de acertos: 193
Porcentagem de acertos: 96.50%

Category: S2NO
KNN: 9
Total de treinamento: 200
Total de testes: 200
Total de acertos: 193
Porcentagem de acertos:

# Resultados

A seguir, está a tabela com a taxa de acerto dos classificadores `KNN` e `SMV`:


Category/KNN | k=5 | K=7 | K=9 |
---- | ---- | ---- | ---- |
**S1** | 97% | 97.5% | `98%`
**S2** | `98%` | `98%` | 97%
**S2NO** | 97% | `99.5%` | 96.5%

Category/SVM | LINEAR |
---- | ---- |
**S1** | 96.5% |
**S2** | 96.5% |
**S2NO** | 94.5% |